In [31]:
import requests
import json
import time
import pandas as pd
import numpy as np

In [4]:
# segments boundaries: split territory on the equal horizontal slices



In [11]:
def make_top_url(bounds, request_id):
    part_1 = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"mapBounds":{'
    part_2 = '"west":{west},"east":{east},"south":{south},"north":{north}'.format(**bounds)
    part_3 = '},"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isPreMarketForeclosure":{"value":false},"isPreMarketPreForeclosure":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"enableSchools":{"value":true}},"isListVisible":true,"mapZoom":9}&wants={"cat1":["listResults","mapResults"],"schoolResults":["schoolResults"]}&requestId='
    
    return part_1 + part_2 + part_3 + str(request_id)

In [12]:
headers_top = {
    'Host': 'www.zillow.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.16; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Cache-Control': 'max-age=0'
}

In [13]:
url_top = make_top_url(bounds[33], 33)
response_top = requests.get(url_top, headers=headers_top)

In [18]:
school_results = response_top.json()['schoolResults']['total']
schools = response_top.json()['schoolResults']['schools']

In [19]:
schools[0]

{'school_id': 42238,
 'name': 'North Star Academy Charter School of Newark',
 'location': {'lat': 40.7422, 'lon': -74.1701},
 'attendance_zones': [200108, 208190],
 'gs_rating': 7,
 'enrollment': 4866,
 'students_per_teacher': 12.11,
 'link': 'https://www.greatschools.org/school?id=03339&state=NJ',
 'grades': 'K-12',
 'is_elementary': True,
 'is_middle': True,
 'is_high': True,
 'is_public': False,
 'is_private': False,
 'is_charter': True}

In [32]:
np.nan

nan

In [37]:
def transform_data(raw_details):
    # function transforms json details to data frame format
    
    raw_details_new = {
        'school_id': raw_details['school_id'] if 'school_id' in raw_details.keys() else np.nan,
        'name': raw_details['name'] if 'name' in raw_details.keys() else np.nan,
        'lat': raw_details['location']['lat'] if 'location' in raw_details.keys() else np.nan,
        'lat': raw_details['location']['lon'] if 'location' in raw_details.keys() else np.nan,
        'attendance_zones': ','.join(str(x) for x in raw_details['attendance_zones']) if 'attendance_zones' in raw_details.keys() else np.nan,
        'gs_rating': raw_details['gs_rating'] if 'gs_rating' in raw_details.keys() else np.nan,
        'enrollment': raw_details['enrollment'] if 'enrollment' in raw_details.keys() else np.nan,
        'students_per_teacher': raw_details['students_per_teacher'] if 'students_per_teacher' in raw_details.keys() else np.nan,
        'link': raw_details['link'] if 'link' in raw_details.keys() else np.nan,
        'grades': raw_details['grades'] if 'grades' in raw_details.keys() else np.nan,
        'is_elementary': raw_details['is_elementary'] if 'is_elementary' in raw_details.keys() else np.nan,
        'is_middle': raw_details['is_middle'] if 'is_middle' in raw_details.keys() else np.nan,
        'is_high': raw_details['is_high'] if 'is_high' in raw_details.keys() else np.nan,
        'is_public': raw_details['is_public'] if 'is_public' in raw_details.keys() else np.nan,
        'is_private': raw_details['is_private'] if 'is_private' in raw_details.keys() else np.nan,
        'is_charter': raw_details['is_charter'] if 'is_charter' in raw_details.keys() else np.nan
    }
    

    #df_l = pd.DataFrame.from_dict(raw_details_new, orient='columns', index=[0], dtype=None, columns=None)
    df_l = pd.DataFrame(raw_details_new, index=[0])
    
    return df_l

In [38]:
transform_data(response_top.json()['schoolResults']['schools'][0])

,school_id,name,lat,attendance_zones,gs_rating,enrollment,students_per_teacher,link,grades,is_elementary,is_middle,is_high,is_public,is_private,is_charter
0,42238,North Star Academy Charter School of Newark,-74.1701,"200108,208190",7,4866,12.11,https://www.greatschools.org/school?id=03339&s...,K-12,True,True,True,False,False,True


In [41]:
df_all = pd.DataFrame()


logs = open('logs.txt', 'w')


broken_segments = []


for segment in bounds.keys():
    url_top = make_top_url(bounds[segment], segment)
    response_top = requests.get(url_top, headers=headers_top)

    if response_top.status_code != 200:
        time.sleep(60)
        response_top = requests.get(url_top, headers=headers_top)

    try:
        if response_top.status_code != 200:
            broken_segments.append(segment)

        else:
            if 'total' in response_top.json()['schoolResults'].keys():
                total_results = response_top.json()['schoolResults']['total']
                received_results = len(response_top.json()['schoolResults']['schools'])
            else:
                total_results = 'unknown'
                
            log_message = """
            Segment: {}, Total objects: {}, Received objects: {}
            """.format(segment, total_results, received_results)
            
            
            logs.write(log_message)
            
            
            for school in response_top.json()['schoolResults']['schools']:
                df_part = transform_data(school)
                
                df_all = df_all.append(df_part)


            print('Cumulative data size: {}'.format(len(df_all)))
            time.sleep(2)

    except:
        continue


        
if len(broken_segments) > 0:
    log_message = """
    Broken Segments: {}
    """.format(', '.join(broken_segments))

    logs.write(log_message)   


logs.close()


df_all.to_csv('schools.csv')

Segment: 1, Total objects: 76, Received objects: 76
Segment: 2, Total objects: 58, Received objects: 58
Segment: 3, Total objects: 72, Received objects: 72
Segment: 4, Total objects: 54, Received objects: 54
Segment: 5, Total objects: 132, Received objects: 132
Segment: 6, Total objects: 124, Received objects: 124
Segment: 7, Total objects: 190, Received objects: 190
Segment: 8, Total objects: 126, Received objects: 126
Segment: 9, Total objects: 173, Received objects: 173
Segment: 10, Total objects: 196, Received objects: 196
Segment: 11, Total objects: 383, Received objects: 200
Segment: 12, Total objects: 57, Received objects: 57
Segment: 13, Total objects: 69, Received objects: 69
Segment: 14, Total objects: 88, Received objects: 88
Segment: 15, Total objects: 115, Received objects: 115


KeyboardInterrupt: 

In [43]:
df_all.school_id.count()

663

In [44]:
df_all.school_id.nunique()

663